In [2]:
import os
import re
import sys
import time
from dotenv import load_dotenv
import google.generativeai as genai
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline
from PIL import Image
import speech_recognition as sr

# LangChain 
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, llms
from langchain.llms.base import LLM
from langchain.schema import Generation, LLMResult, SystemMessage, HumanMessage, AIMessage
from langchain.agents import initialize_agent, Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.llms.base import BaseLLM

# Translator for multilingual 
from googletrans import Translator
from typing import Optional, List

In [ ]:
load_dotenv()
GOOGLE_API_KEY = ""
if not GOOGLE_API_KEY:
    print("Warning: GOOGLE_API_KEY is not set.")
genai.configure(api_key=GOOGLE_API_KEY)
translator = Translator()

In [4]:
class GoogleGenAI(LLM):
    @property
    def _llm_type(self) -> str:
        return "google-generativeai"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.candidates[0].content.parts[0].text

    async def _acall(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self._call(prompt, stop)


In [ ]:
# Regular expression for parsing actions
action_re = re.compile(r'^Action:\s*(\w+)\s*:\s*(.+)$')

# Chatbot class for text-only multi-turn interaction
class Chatbot:
    def __init__(self, system):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result


    def execute(self):
        # Prepare prompt from the conversation history
        prompt = "\n".join([f'{msg["role"]}:{msg["content"]}' for msg in self.messages])
        model = genai.GenerativeModel("gemini-1.5-flash")
        raw_response = model.generate_content(prompt)
        # Extract text content from the first candidate
        result_text = raw_response.candidates[0].content.parts[0].text
        return result_text

In [5]:
google_llm = GoogleGenAI()

In [ ]:




system_prompt = """
You are an expert medical doctor with extensive experience in diagnosing and treating patient queries. Your role is to engage patients in a structured, empathetic, and conversational flow to identify the root cause of their health issues—such as illnesses, aches, or discomforts—and provide tailored solutions. These solutions should include appropriate remedies, medication suggestions (with general disclaimers if needed), lifestyle advice, diet plans, and exercise recommendations to promote overall health and recovery.

For example:

Patient: "I’ve been having a fever."
Doctor: "How long have you been experiencing this fever?"
Patient: "For three days."
Doctor: "Are you also experiencing symptoms like a dry cough, runny nose, or body aches?"
Patient: "No, doctor."
Doctor: "Alright, let’s narrow this down. Have you noticed any other symptoms, like chills, fatigue, or loss of appetite? Also, have you been able to measure your temperature?"
Your responses should:

Ask follow-up questions to gather more details about symptoms, duration, severity, and any relevant medical history.
Analyze the information provided to suggest possible causes (e.g., viral infection, bacterial issue, etc.).
Recommend remedies (e.g., rest, hydration), over-the-counter medications if appropriate (e.g., paracetamol for fever), and caution about consulting a healthcare provider for prescriptions or severe cases.
Provide a simple, practical diet plan to support recovery (e.g., light meals, hydration tips) and address irregular eating habits for long-term health.
Suggest basic exercises or wellness tips (e.g., light stretching or breathing exercises) suitable for their condition.
Maintain a professional yet caring tone, ensuring the patient feels heard and supported.
If information is missing, politely prompt the patient to clarify. Avoid making definitive diagnoses requiring in-person tests, and include disclaimers like ‘Please consult a local doctor if symptoms persist or worsen.’ Ensure all advice aligns with general medical knowledge and promotes a healthy lifestyle.""".strip()

# Action functions using the Gemini API
def generate_workout(level):
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(f"Generate a workout plan for a {level} fitness level")
    return response.candidates[0].content.parts[0].text 

def suggest_meal(preferences):
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(f"Suggest a meal plan with {preferences}")
    return response.candidates[0].content.parts[0].text


# Mapping actions to functions
known_actions = {
    "generate_workout": generate_workout,
    "suggest_meal": suggest_meal,
    
}

# Function for text-based multi-turn conversation
def query(question, max_turns=5):
    i = 0
    bot = Chatbot(system_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print("\nAssistant response:")
        print(result)
        # Look for an action command in the response
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f"\n-- running action: {action} with input: {action_input}")
            observation = known_actions[action](action_input.strip())
            print(f"\nObservation: {observation}\n")
            next_prompt = f"Answer: {observation}"
            # Pause briefly before the next turn
            time.sleep(1)
        else:
            # No further action, so end conversation
            return

# Function to capture voice input from the microphone and return recognized text
def get_voice_query():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    print("Please speak your query now...")
    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        voice_text = recognizer.recognize_google(audio)
        print(f"Recognized voice query: {voice_text}")
        return voice_text
    except sr.UnknownValueError:
        print("Sorry, could not understand the audio.")
    except sr.RequestError as e:
        print(f"Could not request results from the speech recognition service; {e}")
    return None

# Function to handle image queries.
# The user provides a path to an image and an optional accompanying text query.
def handle_image_query():
    image_path = input("Enter the path to the image file: ").strip()
    if not os.path.isfile(image_path):
        print("File not found. Please check the path and try again.")
        return
    try:
        image = Image.open(image_path)
        image.show()  # This will open the image using the default image viewer
    except Exception as e:
        print(f"Error opening image: {e}")
        return

    user_query = input("Enter your query related to this image: ").strip()
    # Upload the file to Google Generative AI (if supported)
    try:
        sample_file = genai.upload_file(path=image_path, display_name="Image")
    except Exception as e:
        print(f"Error uploading image: {e}")
        return

    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
    # Create a prompt that includes both the query and the reference to the uploaded image
    prompt = (f"Using the uploaded image and the query '{user_query}', provide fitness-related advice or suggestions. "
              "The image shows a fitness-related activity or equipment. Assist the user with workout tips, form correction, "
              "or equipment usage guidance.")
    response = model.generate_content([sample_file, prompt])
    print("\nAssistant response for image query:")
    print(response.text)

# Main function to choose the query modality
def main():
    while True:
        print("\nChoose query mode:")
        print("1 - Text query")
        print("2 - Voice query")
        print("3 - Image query")
        print("q - Quit")
        choice = input("Enter your choice: ").strip().lower()

        if choice == "1":
            user_query = input("Enter your text query: ").strip()
            query(user_query)
        elif choice == "2":
            voice_query = get_voice_query()
            if voice_query:
                query(voice_query)
        elif choice == "3":
            handle_image_query()
        elif choice == "q":
            print("Exiting application.")
            sys.exit(0)
        else:
            print("Invalid choice. Please select 1, 2, 3 or q.")

if __name__ == "__main__":
    main()
